In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from typing import Tuple

import sys

In [12]:
df = pd.read_csv("/content/_f048004989fa1185c1d03f0eb2a8ad0c_data-logistic.csv", header=None)
df.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [0]:
y = df.loc[:, 0]
X = df.loc[:, 1:]

In [0]:
def calc_w1 (X: pd.DataFrame, y: pd.DataFrame, w1: float, w2: float, k: float, C: float) -> float:
  l = len(y)
  S = 0
  for i in range(0, l):
    S += y[i] * X[1][i] * (1.0 - 1.0/(1.0 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
  
  return w1 + (S * k) / l - k * C * w1

def calc_w2 (X: pd.DataFrame, y: pd.DataFrame, w1: float, w2: float, k: float, C: float) -> float:
  l = len(y)
  S = 0
  for i in range(0, l):
    S += y[i] * X[2][i] * (1.0 - 1.0/(1.0 + 
                                      np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
  
  return w2 + (k * (1.0 / l) * S) - k * C * w2

In [0]:
def gradient_decent (X: pd.DataFrame, y: pd.DataFrame, w1: float = 0.0, w2: float = 0.0,
          k: float = 0.1, C: float = 0.0, precision: float = 1e-15, max_iter: int = 10000) -> Tuple[float, float]:
  for i in range(0, max_iter):
    w1_prev, w2_prev = w1, w2
    w1, w2 = calc_w1(X, y, w1, w2, k, C), calc_w2(X, y, w1, w2, k, C)
    if np.sqrt((w1_prev - w1) ** 2 + (w2_prev - w2) ** 2) <= precision:
      break
  
  return w1, w2

In [0]:
w1, w2 = gradient_decent(X, y)
w1_reg, w2_reg = gradient_decent(X, y, C=10.0)
# print(w1, w2, w1_reg, w2_reg)

In [25]:
def a(X: pd.DataFrame, w1: float, w2: float) ->  pd.Series:
  return 1.0 / np.exp(-w1*X[1] - w2*X[2])

y_a = a(X, w1, w2)
y_a_reg = a(X, w1_reg, w2_reg)

auc = roc_auc_score(y, y_a)
auc_reg = roc_auc_score(y, y_a_reg)

print(f"{auc:.3f} {auc_reg:.3f}")

0.927 0.936
